In [1]:
# instrumentation
from llama_index.core.instrumentation import get_dispatcher
from llama_index.core.instrumentation.span_handlers import SimpleSpanHandler

# root dispatcher
root_dispatcher = get_dispatcher()

# register span handler
simple_span_handler = SimpleSpanHandler()
root_dispatcher.add_span_handler(simple_span_handler)

In [2]:
# env and langfuse
from dotenv import find_dotenv, load_dotenv
from langfuse import get_client

_ = load_dotenv(find_dotenv())
langfuse = get_client()

from openinference.instrumentation.llama_index import LlamaIndexInstrumentor

# Initialize LlamaIndex instrumentation
LlamaIndexInstrumentor().instrument()

In [3]:
# documents
from llama_index.core import Document

text_list = [
    'Langfuse is an open source LLM engineering platform to help teams collaboratively debug, analyze and iterate on their LLM Applications. '
    'With the Langfuse integration, you can track and monitor performance, traces, and metrics of your LlamaIndex application.' 
    'Detailed traces of the context augmentation and the LLM querying processes are captured and can be inspected directly in the Langfuse UI.',
    
    'Langfuse 真香',
    
    'The instrumentation module (available in llama-index v0.10.20 and later) is meant to replace the legacy callbacks module.',
    
    'Listed below are the core classes as well as their brief description of the instrumentation module: '
    'Event — represents a single moment in time that a certain occurrence took place within the execution of the application’s code.'
    'EventHandler — listen to the occurrences of Event’s and execute code logic at these moments in time.'
    'Span — represents the execution flow of a particular part in the application’s code and thus contains Event’s.'
    'SpanHandler — is responsible for the entering, exiting, and dropping (i.e., early exiting due to error) of Span’s.'
    'Dispatcher — emits Event’s as well as signals to enter/exit/drop a Span to the appropriate handlers.',
]
documents = [Document(text=t) for t in text_list]

In [4]:
# query
from llama_index.llms.openai import OpenAI
llm = OpenAI(model="gpt-5-mini", temperature=0.0)
from llama_index.core import VectorStoreIndex

index = VectorStoreIndex.from_documents(documents)

query_engine = index.as_query_engine(llm=llm)

query = '什麼是 LlamaIndex 的instrumentation module?'
with langfuse.start_as_current_span(name="notebook_test") as span:
    response = query_engine.query(query)
langfuse.flush()

In [5]:
simple_span_handler.print_trace_trees()

SentenceSplitter.__call__-78a09c39-c1d9-4e86-bc18-404c583241e0 (0.003038)
└── SentenceSplitter._parse_nodes-8890127b-43ed-43f8-8578-db5a0e9e56c6 (0.002031)
    ├── SentenceSplitter.split_text_metadata_aware-f4b5200c-8172-4cc3-924f-7c8203425d59 (0.000343)
    ├── SentenceSplitter.split_text_metadata_aware-0cbf998e-c065-4c3e-a890-7558e4f4515a (0.000136)
    ├── SentenceSplitter.split_text_metadata_aware-0c7289c6-ff77-4402-83b2-02e192380417 (0.000137)
    └── SentenceSplitter.split_text_metadata_aware-a29f6a8b-fa08-4635-82f8-dd85c1b900c3 (0.000134)


OpenAIEmbedding.get_text_embedding_batch-daaf2a5d-7b42-43cd-ba85-7f9c59c4608e (0.864471)


RetrieverQueryEngine.query-c0f62d86-71e6-4f9b-8988-720134282db0 (10.357631)
└── RetrieverQueryEngine._query-b8d90ee8-7e2e-4fff-8da1-5a0be526cdf5 (10.356895)
    ├── VectorIndexRetriever.retrieve-94b04d8b-cd8a-4ba2-bfbc-4de7b3b3eabe (0.557083)
    │   └── VectorIndexRetriever._retrieve-8b776ba4-be12-4b91-bd50-bbd01ad4c2fc (0.555986)
    │       └── OpenA

In [7]:
tree.show()

RetrieverQueryEngine.query-c0f62d86-71e6-4f9b-8988-720134282db0 (10.357631)
└── RetrieverQueryEngine._query-b8d90ee8-7e2e-4fff-8da1-5a0be526cdf5 (10.356895)
    ├── CompactAndRefine.synthesize-24d03066-d015-4e59-ab99-c689b6c14ac2 (9.799188)
    │   └── CompactAndRefine.get_response-2ce57c29-d0c3-4d65-a415-5b25bfaa757a (9.796068)
    │       ├── CompactAndRefine.get_response-c21d5fe6-5cfe-451f-a7e8-44fa5f1bcd66 (9.79461)
    │       │   ├── DefaultRefineProgram.__call__-1982f2e8-b194-4aa6-8831-ebd6795073d6 (9.793602)
    │       │   │   └── OpenAI.predict-96fb97a9-ef32-44e6-91e7-d7bb89fa38f7 (9.79317)
    │       │   │       └── OpenAI.chat-e0916b48-41c8-459c-a11f-e295d8362abf (9.79214)
    │       │   └── TokenTextSplitter.split_text-afe832e4-f3b8-461b-ae02-e18df8e8d965 (0.000201)
    │       └── TokenTextSplitter.split_text-6ca3dc15-aec4-4674-ad11-0c58e0d78cc4 (0.000398)
    └── VectorIndexRetriever.retrieve-94b04d8b-cd8a-4ba2-bfbc-4de7b3b3eabe (0.557083)
        └── VectorIndexRetrie

In [20]:
import json
with open('trace.json', 'w') as f:
    json.dump(json.loads(tree.to_json()), f, indent=2, ensure_ascii=False)

In [19]:
tree.to_graphviz()

digraph tree {
	"RetrieverQueryEngine.query-c0f62d86-71e6-4f9b-8988-720134282db0" [label="RetrieverQueryEngine.query-c0f62d86-71e6-4f9b-8988-720134282db0 (10.357631)", shape=circle]
	"RetrieverQueryEngine._query-b8d90ee8-7e2e-4fff-8da1-5a0be526cdf5" [label="RetrieverQueryEngine._query-b8d90ee8-7e2e-4fff-8da1-5a0be526cdf5 (10.356895)", shape=circle]
	"CompactAndRefine.synthesize-24d03066-d015-4e59-ab99-c689b6c14ac2" [label="CompactAndRefine.synthesize-24d03066-d015-4e59-ab99-c689b6c14ac2 (9.799188)", shape=circle]
	"VectorIndexRetriever.retrieve-94b04d8b-cd8a-4ba2-bfbc-4de7b3b3eabe" [label="VectorIndexRetriever.retrieve-94b04d8b-cd8a-4ba2-bfbc-4de7b3b3eabe (0.557083)", shape=circle]
	"CompactAndRefine.get_response-2ce57c29-d0c3-4d65-a415-5b25bfaa757a" [label="CompactAndRefine.get_response-2ce57c29-d0c3-4d65-a415-5b25bfaa757a (9.796068)", shape=circle]
	"VectorIndexRetriever._retrieve-8b776ba4-be12-4b91-bd50-bbd01ad4c2fc" [label="VectorIndexRetriever._retrieve-8b776ba4-be12-4b91-bd50-bbd

In [22]:
!dot -Tpng trace.dot -o trace.png